In [1]:
import os, json, glob, math
import tensorflow as tf
import numpy as np

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
# %matplotlib inline

import pandas as pd

from scipy import signal

In [2]:
import sys
sys.path.append("./pyusct/")
from rfdata import RFdata

In [3]:
dir_dataset = "/mnt/nas/PYUSCT_train/dataset008/"

In [4]:
df = pd.read_csv(os.path.join(dir_dataset, "list.csv"))

In [5]:
df.columns

Index(['id', 'source', 'ix', 'iy', 'x', 'y'], dtype='object')

### 教師データ入出力画像を保存

In [6]:
dataset = np.load(os.path.join(dir_dataset, 'dataset.npz'))

In [7]:
input_all = dataset["input_all"]

In [8]:
N, T, R, L = input_all.shape
N, T, R, L

(10000, 16, 256, 200)

In [9]:
output_all = dataset["output_all"]

In [10]:
N_, H, W = output_all.shape
N_, H, W

(10000, 1, 1)

In [11]:
assert N == N_

R-t画像可視化用関数

In [ ]:
def translate_circular(img):

    num_angle= img.shape[0]
    img_size = int(img.shape[1]*2/np.sqrt(2))

    img_ = np.zeros((img_size, img_size), dtype=img.dtype)

    for i in range(img_size):
        for j in range(img_size):
            i_ = i - img_size //2
            j_ = j - img_size //2

            d = int(np.linalg.norm([i_, j_]))
            if d < img.shape[1]:
                k = ( int(( math.atan2(i_, j_) + np.pi ) // (2*np.pi/num_angle)) )% num_angle

                img_[i, j] = img[k, d]
    
    return img_

In [15]:
dir_save = os.path.join(dir_dataset, 'image')
if not os.path.exists(dir_save): os.makedirs(dir_save)
    
sourcepath = ""
rf = None

for i in range(len(df)):
# for i in range(5):
    
    input_sample  = input_all[i,:,:,:]
    output_sample = output_all[i,:,:]
    img_Treduced = np.mean(input_sample, axis=0)
    ts = np.mean(img_Treduced, axis=0)*T*R
    
    src = df.at[i, 'source']
    ix = df.at[i, 'ix']
    iy = df.at[i, 'iy']
    
    print((i, iy, ix), flush=True)

    if(sourcepath != src):
        sourcepath = src
        rf = RFdata(src)
        sct_mean = np.mean(rf.medium_sct.flatten())
        
    _vmax = 0.005
    _vmin = np.power(0.005, 2)
    
    plt.clf()
    
    fig = plt.figure(figsize=(20,20))
    
#     plt.subplot(2,3,1)
#     H,W = sa_sample.shape
#     plt.imshow(sa_sample, interpolation='nearest', cmap='gray', vmin=_vmin*T*R, vmax=_vmax*T*R/4)
#     plt.title('SA {0} {1}'.format(sa_sample.shape, sa_sample[H//2,W//2]))
    
    plt.subplot(2,3,1)
    plt.imshow(img_Treduced, interpolation='nearest', cmap='hot', vmin=_vmin, vmax=_vmax)
    plt.title('RF.mean(T)')
    
#     plt.subplot(2,3,3)
#     plt.imshow(translate_circular(img_Treduced), interpolation='nearest', cmap='hot', vmin=_vmin, vmax=_vmax)
#     plt.title('RF.mean(T) circular')
    
    plt.subplot(2,2,2)
    plt.plot(ts)
    plt.plot( np.array([len(ts)//2]*2), np.array([_vmin, ts[len(ts)//2]]), color='red')
    plt.ylim(_vmin*T*R, _vmax*T*R/4)
    plt.title('RF.mean(T,R)')

    plt.subplot(2,2,3)
    img_whole = np.copy(rf.medium_sct)
    linecolor = 2*np.max(img_whole)
    img_whole[iy-1:iy+2,:] = linecolor
    img_whole[:,ix-1:ix+2] = linecolor
    plt.imshow(img_whole, interpolation='nearest')
    plt.title('scattering magnitude')

    plt.subplot(2,2,4)
    #plt.imshow(output_sample, interpolation='nearest', vmin=0, vmax=sct_mean)
    plt.imshow(output_sample, interpolation='nearest', vmin=0, vmax=1, cmap='gray')
    plt.title('scattering magnitude')

    fig.savefig(os.path.join(dir_save, '{0:0>8}.png'.format(i)))
       

(0, 707, 439)


/root/.pyenv/versions/anaconda3-4.1.1/lib/python3.5/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


(1, 531, 319)
(2, 574, 639)
(3, 550, 589)
(4, 385, 543)
(5, 625, 355)
(6, 452, 511)
(7, 700, 502)
(8, 699, 461)
(9, 554, 327)
(10, 391, 538)
(11, 456, 342)
(12, 548, 615)
(13, 566, 545)
(14, 656, 614)
(15, 713, 735)
(16, 574, 360)
(17, 532, 640)
(18, 656, 547)
(19, 705, 530)
(20, 352, 347)
(21, 662, 534)
(22, 424, 524)
(23, 726, 374)
(24, 536, 740)
(25, 490, 683)
(26, 390, 551)
(27, 474, 643)
(28, 491, 522)
(29, 661, 326)
(30, 494, 366)
(31, 353, 359)
(32, 531, 661)
(33, 475, 624)
(34, 604, 562)
(35, 704, 599)
(36, 681, 528)
(37, 718, 332)
(38, 358, 708)
(39, 641, 682)
(40, 418, 526)
(41, 357, 522)
(42, 558, 646)
(43, 574, 443)
(44, 403, 291)
(45, 636, 481)
(46, 441, 499)
(47, 689, 650)
(48, 418, 284)
(49, 386, 600)
(50, 514, 469)
(51, 602, 480)
(52, 453, 276)
(53, 503, 691)
(54, 405, 531)
(55, 318, 730)
(56, 714, 546)
(57, 646, 566)
(58, 609, 748)
(59, 662, 381)
(60, 558, 585)
(61, 408, 698)
(62, 443, 436)
(63, 609, 482)
(64, 518, 291)
(65, 539, 578)
(66, 721, 490)
(67, 349, 675)
(68,

KeyboardInterrupt: 

In [ ]:
rf.medium_sct[568,733]

## SAの性能評価

In [ ]:
arr_sa = []
arr_sct = []

for i in range(len(df)):
    
    input_sample = np.load(input_dataset[i])
    output_sample = np.load(output_dataset[i])

    ts = np.log(np.sum(np.sum(input_sample, axis=0), axis=0)+np.power(0.005, 2))
    arr_sa.append(ts[len(ts)//2])
    
    H, W = output_sample.shape
    arr_sct.append(output_sample[H//2, W//2])


In [ ]:
plt.clf()
plt.scatter(np.array(arr_sa), np.array(arr_sct))
fig.savefig('./sa_quality.png')